# Day 1 of ML

- [ ] how to do: preprocessing (with `ColumnTransformer`)
- [ ] how to examine: preprecessing
- [ ] how to do: pipelines + cross validation + scoring
    - `make_pipeline()` with preprocessing and estimator
    - examine pipeline elements 
    - fit and predict using the pipeline (not CV)
    - examine the pipeline using CV (using the cv function and examining its output)
- [ ] scoring vocab: recall/sensitivity, precision, specificity, accuracy, 
- [ ] how to do: optimizing a pipeline by "tuning the hyperparameters"
    - hyperparameters are the parameters of functions/estimators in the steps in your pipeline
    - set up and use `gridsearchCV`
    - examine output of `gridsearchCV`
    

In [1]:
from sklearn import set_config
set_config(display="diagram")  # display='text' is the default

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline 
from sklearn.impute import SimpleImputer
from df_after_transform import df_after_transform
from sklearn.model_selection import KFold, cross_validate, GridSearchCV

pd.set_option('display.max_colwidth', 1000, 'display.max_rows', 50, 'display.max_columns', None) 

## Load data 

In [2]:
loans = pd.read_csv('inputs/2013_subsample.zip')

## Create the training and holdout samples

Split your data into test and train. Your options:
- [sklearn has some built in splitters](https://scikit-learn.org/stable/modules/cross_validation.html)
- [`test_train_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) can do basic splits, but may not be appropriate
    - _test_ is the typical sklearn vernacular, on the website I call this this holdout sample
- You can just keep the most recent time period of your samples in the holdout and put the rest into the training data.

In [3]:
# split to test and train (what we call the "test" subset here is the "holdout" data)

# first let's separate y from X (as is typically done)
y = loans.loan_status == 'Charged Off'
y.value_counts()
loans = loans.drop('loan_status',axis=1)

# stratify will make sure that test/train both have equal fractions of outcome
X_train, X_test, y_train, y_test = train_test_split(loans, y, stratify=y, test_size=.2, random_state=0)

## EDA

On the **TRAINING DATA ONLY**: 
- do lots of EDA
- look for missing values, which variables are what type, and outliers 
- figure out how you'd clean the data (imputation, scaling, encoding categorical vars)
- these lessons will go into the preprocessinG portion of your pipeline 
- PRO TIP: `pandas-profiling` and `dabl` build automated reports. Which is nice, but remember you need to examine them closely! **There is no shortcut for EDA.**

In [4]:
# from pandas_profiling import ProfileReport
# 
# profile = ProfileReport(pd.concat([y_train, X_train], axis=1), 
#                         title='Lending Club Profiling Report',
#                         html={'style':{'full_width':True}}) 
# profile.to_file("inputs/lending_club_EDA_training.html") # can take a minute or two with this dataset size. Let's look at the one I uploaded...


Individual loans, k = 34, N = 108k, 

y is Loan default, 15% default

Maybe useful X variables
- 2 fico score variables?!
- bankruptcy
- int rate
- grade 
- dti, loan ammount, annual income
- term

Some of these are categorical! 

## NOW LET'S LEARN EACH PART OF "Optimize a series of models" FROM THE TEMPLATE

### Steps 1 and 2: Preprocessing


In [5]:
# set up pipeline to clean each type of variable (1 pipe per var type)

numer_pipe = make_pipeline(SimpleImputer(strategy='mean'),
                           StandardScaler()) 

cat_pipe   = make_pipeline(OneHotEncoder(drop='first'))

# combine those pipes into "preprocess" pipe

preproc_pipe = ColumnTransformer(  
    [ # arg 1 of ColumnTransformer is a list, so this starts the list
    # a tuple for the numerical vars: name, pipe, which vars to apply to
    ("num_impute", numer_pipe, ['annual_inc','int_rate']),
    # a tuple for the categorical vars: name, pipe, which vars to apply to
    ("cat_trans", cat_pipe, ['grade'])
    ]
    , # ColumnTransformer can take other args, most important: "remainder"
    remainder = 'drop' # you either drop or passthrough any vars not modified above
)



_Note: You can have multiple "numeric pipes" or "cat pipes" if you think some number variables should receive different treatments, which is very common!_

In [6]:
###########
# hot tip: check out what this preprocessing does before you continue!
###########

preproc_df = df_after_transform(preproc_pipe,X_train)
print(f'There are {preproc_df.shape[1]} columns in the preprocessed data.')
preproc_df.describe().T.round(2)

There are 8 columns in the preprocessed data.


C:\Users\DonsLaptop\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,count,mean,std,min,25%,50%,75%,max
annual_inc,107843.0,-0.00,1.00,-1.36,-0.55,-0.19,0.32,121.55
int_rate,107843.0,-0.00,1.00,-1.92,-0.76,-0.05,0.68,2.60
grade_B,107843.0,0.33,0.47,0.00,0.00,0.00,1.00,1.00
grade_C,107843.0,0.28,0.45,0.00,0.00,0.00,1.00,1.00
grade_D,107843.0,0.15,0.36,0.00,0.00,0.00,0.00,1.00
grade_E,107843.0,0.07,0.25,0.00,0.00,0.00,0.00,1.00
grade_F,107843.0,0.03,0.18,0.00,0.00,0.00,0.00,1.00
grade_G,107843.0,0.01,0.08,0.00,0.00,0.00,0.00,1.00


- Standard scaler makes a variable mean 0 std 1
- OneHotEncoder takes a categorical variable, and turns it into "C" (the # of cats) binary variables
- crazy outliers in annual_inc

### EXERCISES

1. How many observations for `annual_inc` were non-missing before the processing? Does our imputation choice here we make a small or large impact?
1. How many values of `grade` was there in the data before and after `preproc_pipe`? 
1. Above, revise `preproc_pipe` to include another continuous variable.

In [7]:
# answers here
#1 - no blanks, imputation didnt matter for that variable!
#2 - 7 values before, 6 after - bc of "drop=first". whether you drop first dpends on model (reg = drop)
#3 

### Prof demo: Fitting and using ONE model

Warning: This is not best practice to do on the whole training sample. The point here is to simply show you how to estimate and use a model.

_(The only time you fit a model on the whole training sample is the VERY end of the template, right before you check to see how it does on the holdout.)_

Steps: https://ledatascifi.github.io/ledatascifi-2022/content/05/04c_onemodel.html

To fit the model: `<model>.fit(X,y)`

In [8]:
# create ("instantiate") the estimator class with some hyperparameters,
# note: the hypermeters are whatever is inside the "()"
# assign this instance of the estimator to an object
logit = LogisticRegression(C=.05)

# fit the model to the data: <model>.fit(X,y)
# note: I'm only using annual income here for illustration
logit.fit(X_train[['annual_inc']], 
          y_train) 

LogisticRegression(C=0.05)

To use the model: `<model>.predict(X)`

In [9]:
# this creates predicted values 
y_pred = logit.predict(X_train[['annual_inc']],)

print(f'''
% predicted as charge offs: {round(100*(y_pred == 1).mean(),2)}
Accuracy:                   {round(100*(y_pred == y_train).mean(),2)}
''')


% predicted as charge offs: 0.0
Accuracy:                   84.4



### EXERCISES

- Q4: Let's estimate a different logit model, and see if the accuracy or the number of predictions of charge offs changes. [This time change the penalty OR the value of "C".](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)

In [10]:
# answers here

# the parameters inside the pipeline's functions are called "hyperparameters"

### Steps 3, 4, and 5: Pipelines, CV, and model evaluation (scoring)

Making a pipeline is easy: `make_pipeline` will put steps together for you:

In [11]:
logit_pipe = make_pipeline(preproc_pipe, LogisticRegression())

A pipeline is an object that stores its steps (and steps within steps)

In [12]:
logit_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_impute',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['annual_inc', 'int_rate']),
                                                 ('cat_trans',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['grade'])])),
                ('logisticregression', LogisticRegression())])

Fitting the model and using it is easy:

In [13]:
logit_pipe.fit(X_train, y_train) 
y_pred = logit_pipe.predict(X_train)

print(f'''
% predicted as charge offs: {round(100*(y_pred == 1).mean(),2)}
Accuracy:                   {round(100*(y_pred == y_train).mean(),2)}
''')


% predicted as charge offs: 0.0
Accuracy:                   84.4



But the better idea is to see how that model does on different "folds" of the data: CROSS VALIDATION.

The [`cross_validate` (function docs here)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html?highlight=cross_validate#sklearn.model_selection.cross_validate) makes this easy:

In [14]:
scores = cross_validate(logit_pipe,
                        X_train, y_train,
                        cv = 10,
                        scoring=['precision','recall'])

C:\Users\DonsLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DonsLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DonsLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DonsLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetri

In [15]:
scores

{'fit_time': array([0.33701301, 0.32693172, 0.31268835, 0.31123757, 0.23871469,
        0.32928491, 0.3208344 , 0.33399677, 0.27784681, 0.33608222]),
 'score_time': array([0.01677871, 0.01806784, 0.01670718, 0.01438403, 0.01973248,
        0.01898098, 0.01803422, 0.01592422, 0.01619911, 0.01587272]),
 'test_precision': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'test_recall': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}

### EXERCISES

- Q5: What is the average score of this model in the folds?
- Q6: How many folds did we use above (by default). Change to 10 folds and repeat.
- Q7: What is the "score" being reported? 
- Q8: Without running code, [what do you think our model is currently scoring for precision, sensitivity, and recall?]
- Q9: Change the scoring method to one of those. [Specify them in sklearn is here.](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter). How did your score change and why?


In [16]:
# q5
scores['test_recall'].mean()

0.0

In [17]:
# q6

# done, adj code above,
# adding more # folds is easy, but "more" may not be what you want
# if outcome is NOT balanced (50/50,33/33/33,...)
# "STRATIFY" YOUR FOLDS

### q7, q8

- sensitivity/recall: the fraction of a given outcome that you find
    - \# of obs you call true that are true / # of actually true    (does the model find defaults?)
        - TP / # of actually true
    - \# of obs you call false that are false / # of actually false  (does the model find paybacks?)
        - TN / # of actually false
- precision: how accurate is your predictions of a given outcome?
    - \# of obs you call true that are true / # you call true
        - TP / # you call true 
    - \# of obs you call false that are false / # you call false
        - TN / # you call false
        
Here:
- Recall of defaults is 0: It never predicts default!
- Recall of payback is > 0
- Precision of default predictions: inf or divide by zero
- Precision of payback predictions: 84%

### Step 6: Optimizing the model

### Aka "tuning the hyperparameters"

It seems like something about how the default values of the logit model are set is leading to a simpleton model: Predict every loan is paid back!

So, the idea here is to repeat the CV above, many times, with different parameter values.


### What hyperparameters can I change?

The easiest way to see all the hyperparameters in the pipeline is this: 

`<pipename>.get_params()`

Notice how the "C" parameter for the LogisticRegression function is called "`logisticregression__C`" below? We will come back to that in  a second!

In [18]:
# logit_pipe.get_params()

### Setting up the "hyperparameter grid"

The combination of parameters you want to try is
- a dictionary
- the **keys** in the dict are the hyperparameters you want to change (specifically, how the parameter is **named in the pipeline**
- the **values** in the dict are the values for that hyperparameter you want to change

For example:
```python
parameters =  {'logisticregression__C': [0.001,0.1,1,5]}
```

The reason I wrote the weird "`logisticregression__C`", is because this will help `sklearn` find the function and its parameter to change. This means that within the called "logisticregression" (followed by two underscores) there is a parameter called "C".

Similarly, within the "columntransformer" step, there is a "num_impute" step, which has a "simpleimputer" step, which is a function that has a parameter called "strategy".

Thus, if you want to try other strategies for filling blank numbers in, "`'columntransformer__num_impute__simpleimputer__strategy'`" is what you need.

Thus:
```python
parameters =  {'logisticregression__C': [0.001,0.1,1,5],
              'columntransformer__num_impute__simpleimputer__strategy' : ['mean','median']}
```


### All together now...

1. Set up your parameter grid
1. Make a "super estimator" object with `GridSearchCV`. This just runs cross_validate for every combination of parameters in the grid. (Below, 2x3=6 combinations.)
1. Just like CV, use `.fit()` to run the grid search.

In [19]:
# set up hyper param grid - what params in a pipeline do you want to change?
# a dictionary. keys are things to change in pipeline, values are what to try for that param
# key: <stepname>__<parametername>

parameters =  {'logisticregression__C': [0.1,1,5], 
              'columntransformer__num_impute__simpleimputer__strategy' : ['mean','median']}

#     find optimal hyper params (gridsearchcv)

grid_search = GridSearchCV(estimator = logit_pipe, 
                           param_grid = parameters,
                           cv = 5,
                           scoring = ['accuracy','precision','recall','f1'],
                           refit = 'recall'
                           )

grid_search

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num_impute',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['annual_inc',
                                                                          'int_rate']),
                                                                        ('cat_trans',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder(drop='first'))]),
                                                                         ['grade'])])),
                                       ('logisticregression',
                                        LogisticRegression())]),
             param_grid={'columntransformer__num_impute__simpleimputer__strategy': ['mean',
                                                                                    'median'],
                         'logisticregression__C': [0.1, 1, 5]},
             refit='recall', scoring=['accuracy', 'precision', 'recall', 'f1'])

In [20]:
results = grid_search.fit(X_train,y_train)

#     save pipeline with optimal params in place
#     (Note: you should spend time interrogating model predictions, plotting and printing.
#     Does the model struggle predicting certain obs? Excel at some?)

C:\Users\DonsLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DonsLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DonsLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DonsLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetri

In [21]:
#q10

# 1 row for each possible combo of parameters
pd.DataFrame(results.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_columntransformer__num_impute__simpleimputer__strategy,param_logisticregression__C,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,0.282212,0.031528,0.035546,0.002037,mean,0.1,"{'columntransformer__num_impute__simpleimputer__strategy': 'mean', 'logisticregression__C': 0.1}",0.844035,0.844035,0.844035,0.844028,0.844075,0.844042,0.000017,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.260060,0.025242,0.033931,0.003858,mean,1,"{'columntransformer__num_impute__simpleimputer__strategy': 'mean', 'logisticregression__C': 1}",0.844035,0.844035,0.844035,0.844028,0.844075,0.844042,0.000017,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.248491,0.019128,0.031696,0.001700,mean,5,"{'columntransformer__num_impute__simpleimputer__strategy': 'mean', 'logisticregression__C': 5}",0.844035,0.844035,0.844035,0.844028,0.844075,0.844042,0.000017,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.256401,0.025159,0.032600,0.004842,median,0.1,"{'columntransformer__num_impute__simpleimputer__strategy': 'median', 'logisticregression__C': 0.1}",0.844035,0.844035,0.844035,0.844028,0.844075,0.844042,0.000017,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.273799,0.031244,0.032200,0.003370,median,1,"{'columntransformer__num_impute__simpleimputer__strategy': 'median', 'logisticregression__C': 1}",0.844035,0.844035,0.844035,0.844028,0.844075,0.844042,0.000017,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5,0.276002,0.029576,0.034000,0.003037,median,5,"{'columntransformer__num_impute__simpleimputer__strategy': 'median', 'logisticregression__C': 5}",0.844035,0.844035,0.844035,0.844028,0.844075,0.844042,0.000017,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [22]:
best_logit = results.best_estimator_

### What to do after that?

[The "outputs" of the grid search are the attributes of the results object, as listed here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearchcv#sklearn.model_selection.GridSearchCV)

- Q10: Output the CV results
    - Bonus: Do it as a dataframe
- Q11: Make the grid search output recall, sensitivity, f1, and accuracy 

---

Now, here is what I'd _**like**_ to ask you:

- Q12: Which of these models would you choose, taking into account the bias-variance tradeoff? Discuss whether these models are high bias or not, and whether they are high variance or not.
- Q13: Outline how we might adjust our model here to improve its performance

**Except: Don't answer those for today's (bad) logit model. I've "hidden" something from you about why this model is performing so poorly. Think of this as "a case study in miniature" about how "black boxes" can make dealing with ML models impenetrable.**

---


In [23]:
# put answers here for Q10 and Q11